# Recommender of Recommenders

Using AI Reasoning Models to recommend reco algorithms.

### Step 1: Setup and initialization

In [2]:
# Automatically reload modules when they change
%load_ext autoreload
%autoreload 2  

# Import necessary libraries and load environment variables
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file (contains API keys for OpenAI/Azure OpenAI)

# Import data modeling and typing libraries
from pydantic import BaseModel
from typing import List, Dict, Union
from typing import Any
from pathlib import Path
import re
import json

# Add the current directory to the Python path to allow imports from local modules
import sys
sys.path.append('.')

# Import utility functions and models
from utils.file_utils import *      # File handling utilities
from utils.openai_utils import *     # OpenAI API interaction utilities
from utils.openai_data_models import *  # Data models for API interactions
from master_reco.master_reco_models import *  # Models specific to recommendation system

# Setup rich console for better output formatting
from rich.console import Console
console = Console()

# Set up directory path and model configuration
module_directory = os.path.dirname(".")

# Configure the OpenAI model to use the "o3" model with high reasoning
# o3 is a reasoning-focused model that performs well for complex analysis tasks
model_info = TextProcessingModelnfo(
    model_name="o3",             
    reasoning_efforts="high",  # High reasoning effort for detailed analysis   
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 2: Define the business scenario for recommendation system analysis

In [3]:
# This scenario description will be used as input to the AI model for analysis
scenario = """A major task in applying recommendations in retail is to predict which products or set of products a user is most likely to engage with or purchase, based on the shopping or viewing history of that user. This scenario is commonly shown on the personalized home page, feed or newsletter. 

### You might also like

In this scenario, the user is already viewing a product page, and the task is to make recommendations that are relevant to it.  Personalized recommendation techniques are still applicable here, but relevance to the product being viewed is of special importance.  As such, item similarity can be useful here, especially for cold items and cold users that do not have much interaction data.

### Frequently bought together

In this task, the retailer tries to predict product(s) complementary to or bought together with a  product that a user already put in to shopping cart. This feature is great for cross-selling and is normally displayed just before checkout.  In many cases, a machine learning solution is not required for this task.

### Similar alternatives

This scenario covers down-selling or out of stock alternatives to avoid losing a sale. Similar alternatives predict other products with similar features, like price, type, brand or visual appearance.

## Other considerations

Retailers use recommendation to achieve a broad range of business objectives, such as attracting new customers through promotions, or clearing products that are at the end of their season. These objectives are often achieved by re-ranking the outputs from recommenders in scenarios above.

"""
# The scenario covers multiple retail recommendation use cases:
# 1. Personalized product recommendations based on user history
# 2. "You might also like" recommendations on product pages
# 3. "Frequently bought together" for cross-selling
# 4. "Similar alternatives" for out-of-stock items

### Step 3: Generate algorithm recommendations based on the scenario

In [4]:
# Step 1: Load the prompt template for recommendation algorithm selection
reco_selection_prompt_file = locate_prompt("reco_selection.txt", module_directory)
reco_selection_template = read_file(reco_selection_prompt_file)

# Step 2: Format the prompt template with our scenario
reco_selection_prompt = reco_selection_template.format(scenario=scenario)

# Step 3: Call the OpenAI model to analyze the scenario and recommend algorithms
# The response will be structured according to the RankedAlgosResponse model
reco_selection = call_llm_structured_outputs(
    reco_selection_prompt,  # The formatted prompt
    model_info=model_info,  # Model configuration (o3 with high reasoning)
    response_format=RankedAlgosResponse  # Expected response structure
)

# Step 4: Display the recommended algorithms with nice formatting
console.print("\n[bold green]Recommender Selection[/bold green]")
console.print(reco_selection)
# The output will contain ranked recommendation algorithms with explanations
# for why each is suitable for the described retail scenario

Reading file from path: c:\Users\selhousseini\Documents\GitHub\recommenders\examples\07_tutorials\ai-reco-of-recos\prompts\reco_selection.txt
>>>>> https://dev-aoai-swedencentral.openai.azure.com

call_llm_structured_o3::Calling OpenAI APIs with 1 messages - Model: o3 - Endpoint: https://dev-aoai-swedencentral.openai.azure.com/openai/



Recommender Selection

RankedAlgosResponse(
    ranked_algos=[
        RankedAlgo(
            name='SAR',
            why='This item-item similarity model excels on implicit events, letting the retailer pre-compute 
“Frequently Bought Together” and context-aware “You Might Also Like” lists with millisecond latency and no heavy 
training. SAR scales to large SKU catalogs and still works when user or item history is sparse because it relies on
simple co-occurrence counts.'
        ),
        RankedAlgo(
            name='LightFM',
            why='LightFM fuses collaborative signals with product metadata (price, brand, text), so it keeps 
recommendations relevant to the product being viewed while mitigating cold-start for new items or users. The hybrid
design makes it a strong fit for both personalized home-page feeds and “Similar Alternatives” when the original 
item is out of stock.'
        ),
        RankedAlgo(
            name='LightGBM-GBT',
            why='As a second-stage re-ranker over candidates from SAR/LightFM, LightGBM can directly optimize 
business KPIs such as CVR or margin and inject promotion or clearance rules. It offers fast CPU inference (<100 ms)
and interpretable feature importance, satisfying real-time retail constraints and merchandising teams alike.'
        )
    ]
)

### Step 4: Generate a detailed implementation plan based on scenario and algorithm recommendations

In [ ]:
# Step 1: Load the prompt template for the action plan
action_plan_prompt_file = locate_prompt("action_plan.txt", module_directory)
action_plan_template = read_file(action_plan_prompt_file)

# Step 2: Format the prompt with both the scenario and the recommended algorithms
# This allows the model to create a plan specifically for the selected algorithms
action_plan_prompt = action_plan_template.format(
    scenario=scenario, 
    algos=str(reco_selection.model_dump())  # Convert the algorithm recommendations to a string
)

# Step 3: Call the OpenAI model to generate a detailed implementation plan
# The response will be structured according to the RecommendationDeploymentPlan model
plan_of_action = call_llm_structured_outputs(
    action_plan_prompt, 
    model_info=model_info,
    response_format=RecommendationDeploymentPlan  # Expected response structure
)

# Step 4: Display the implementation plan with nice formatting
console.print("\n[bold green]Plan of Action[/bold green]")
console.print(plan_of_action)
# The output will contain a comprehensive implementation plan including:
# - Candidate recommendation generators
# - Re-ranking strategies
# - Training infrastructure requirements
# - Serving architecture on Azure
# - Evaluation metrics and rollout strategy
# - Documentation and Infrastructure as Code specifications

Reading file from path: c:\Users\selhousseini\Documents\GitHub\master_reco\prompts\action_plan.txt

call_llm_structured_o3::Calling OpenAI APIs with 1 messages - Model: o3 - Endpoint: https://dev-aoai-swedencentral.openai.azure.com/openai/



Plan of Action

RecommendationDeploymentPlan(
    candidate_generators=[
        CandidateGenerator(
            model='LightGCN graph recall (30-day user–item bipartite)',
            data_needed='user_id,item_id, implicit events (view, click, add-to-cart) last 30 days in Delta Lake; 
item metadata for cold-start fallback',
            batch_cadence='nightly'
        ),
        CandidateGenerator(
            model='BM25 + Azure OpenAI embedding hybrid on item title/description',
            data_needed='product_id, title, description, category taxonomy; OpenAI text-embedding-ada-002 vectors 
stored in Azure Cognitive Search index',
            batch_cadence='hourly incremental'
        ),
        CandidateGenerator(
            model='Time-decayed top-Pop (sliding 24 h)',
            data_needed='aggregated item interaction counts from Azure Stream Analytics over Event Hub stream',
            batch_cadence='near-real-time (5-min micro-batches)'
        )
    ],
    reranker=ReRanker(
        architecture='xDeepFM (ID embeddings + dense price/brand/ctx features)',
        loss='List-wise LambdaRank with label-gain 2^(click_depth)-1',
        label_definition='Positive = session click ≥8 s dwell OR add-to-cart within 24 h; negatives = sampled 
unclicked impressions',
        latency_target='<30 ms per 50 candidates on AKS GPU'
    ),
    training_stack=TrainingStack(
        etl='PySpark on Azure Databricks → Delta Lake bronze/silver/gold',
        framework='PyTorch Lightning + TorchRec',
        hpo_strategy='AzureML HyperDrive Bayesian optimisation (20 trials, early-terminate via ASHA)',
        hardware='1×Standard_NC24ads_A100_v4 (4 × A100 80 GB) + 4 vCPU driver'
    ),
    serving_path=ServingPath(
        storage='Redis Enterprise (Azure Cache) for online features; models in Azure ML Managed Endpoint blob 
store',
        ann_layer='Faiss HNSW (vector recall) hosted in AKS sidecar',
        model_runtime='ONNX-Runtime with TensorRT on AKS',
        p99_latency='≤50 ms end-to-end (recall 20 ms + re-rank 30 ms)'
    ),
    metrics_rollout=MetricsRollout(
        offline_metrics=['nDCG@10', 'MAP@20', 'Recall@50', 'Coverage'],
        online_kpis=['CTR', 'Add-to-Cart Rate', 'Revenue per Session'],
        rollout_strategy='Shadow for 48 h ➔ Canary 1% ➔ 5% ➔ 25% ➔ 100% over 3 weeks',
        testing='Frequentist A/B with sequential testing; holdback 10% control'
    ),
    docs_iac=DocsIac(
        artifacts=[
            'README.md',
            'Architecture_diagram.drawio',
            'MLflow experiment reports',
            'Grafana dashboards',
            'Run-book.md'
        ],
        iac_spec='Bicep modules in /infra deploying AKS, Redis, Cognitive Search, AML Workspace',
        monitoring='Azure Monitor + Application Insights + Prometheus exporters'
    )
)